# Introduction to Mapping 

In this lesson we're going to be learning how to analyze and visualize geographic data.

We'll be using two libraries, GeoPy and Folium.

[GeoPy](https://geopy.readthedocs.io/en/stable/) is a Python library makes it easier to use a range of third-party geocoding API services, such as Google, Bing, ArcGIS, and OpenStreetMap to generate geographic coordinates.

[Folium](https://python-visualization.github.io/folium/) is a library that integrates with the open-source JavaScript mapping software, [Leaflet.js](https://leafletjs.com/) to create interactive maps.

### Install GeoPy

In [1]:
!pip install geopy

### Import Nominatim modules

In [2]:
from geopy.geocoders import Nominatim

Nominatim (which means “name” in Latin) uses OpenStreetMap data to match addresses with geopgraphic coordinates. Though we don’t need an API key to use Nominatim, we do need to create a unique application name.

### Initializing Nominatim 
Here we're initializing Nominatim as a variable called `geolocator` Change the application name below to your own application name.

In [11]:
geolocator = Nominatim(user_agent="YOUR NAME's mapping app", timeout=2)

## Geocode a location
To geocode an address or location, we simply use the `.geocode()` function:

In [12]:
location = geolocator.geocode("Nassau Hall")

In [13]:
location

Location(Nassau Hall, Chapel Drive, Princeton, Mercer County, New Jersey, 08542, United States, (40.34868635, -74.65939173555758, 0.0))

### Get location address

In [14]:
print(location.address)

Nassau Hall, Chapel Drive, Princeton, Mercer County, New Jersey, 08542, United States


### Get Latitude and Longitude

In [15]:
print(location.latitude, location.longitude)

40.34868635 -74.65939173555758


### Get Location Class

In [16]:
print(f"Class: {location.raw['class']} \nType: {location.raw['type']}")

Class: building 
Type: university


### Retrieve multiple possible matches

In [22]:
possible_locations = geolocator.geocode("Nassau Hall", exactly_one=False)

for location in possible_locations:
    print(location.address)
    print(location.latitude, location.longitude)
    print(f"Importance: {location.raw['importance']}")

Nassau Hall, Chapel Drive, Princeton, Mercer County, New Jersey, 08542, United States
40.34868635 -74.65939173555758
Importance: 0.5681802197739116
Nassau Hall, Mary Jemison Drive, Geneseo, Livingston County, New York, 14454, United States
42.7922854 -77.82412352252632
Importance: 0.201
Nassau Hall, Marburger Drive, Suffolk County, New York, 11794-5250, United States
40.90611495 -73.12092235439
Importance: 0.201


## Geocoding with Pandas

To geocode every location in a CSV file, we can use Pandas, make a Python function, and `.apply()` it to every row in the CSV file.

In [27]:
import pandas as pd
pd.set_option("max_rows", 400)
pd.set_option("max_colwidth", 400)

Here we make a function with `geolocator.geocode()` and ask it to return the address, lat/lon, and importance score:

In [28]:
def find_location(row):
    
    place = row['place']
    
    location = geolocator.geocode(place)
    
    if location != None:
        return location.address, location.latitude, location.longitude, location.raw['importance']
    else:
        return "Not Found", "Not Found", "Not Found", "Not Found"

### Let's read in a CSV with a list of places in Princeton

In [30]:
princeton_df = pd.read_csv("../_datasets/princeton-places.csv")

In [31]:
princeton_df

,place
0,Nassau Hall
1,Community Park North
2,Firestone Library
3,Princeton Art Museum
4,The Bent Spoon
5,Nomad Pizza
6,Princeton Junction station
7,Mamoun's Falafel
8,"Palmer Square, Princeton"
9,Marquand Park


Now let’s `.apply()` our function to this Pandas dataframe and see what results Nominatim’s geocoding service spits out. Run the cell below.

In [71]:
princeton_df[['address', 'lat', 'lon', 'importance']] = princeton_df.apply(find_location, axis="columns", result_type="expand")
princeton_df

,place,address,lat,lon,importance
0,Nassau Hall,"Nassau Hall, Chapel Drive, Princeton, Mercer County, New Jersey, 08542, United States",40.348686,-74.659392,0.568180
1,Community Park North,"Community Park North, Andrews Lane, North Princeton, Princeton, Mercer County, New Jersey, 08540, United States",40.361364,-74.669650,0.301000
2,Firestone Library,"Firestone Library, 1, Washington Road, Princeton, Mercer County, New Jersey, 08544, United States",40.349649,-74.657297,0.201000
3,Princeton Art Museum,"Art Museum, Elm Drive, Princeton, Mercer County, New Jersey, 08544, United States",40.346995,-74.657635,0.311000
4,The Bent Spoon,"The Bent Spoon, Palmer Square West, Princeton, Mercer County, New Jersey, 08542, United States",40.349881,-74.661851,0.301000
5,Nomad Pizza,"Nomad Pizza, East Broad Street, Hopewell, Mercer County, New Jersey, 08525, United States",40.389748,-74.761602,0.201000
6,Princeton Junction station,"Princeton Junction, Plainsboro Township, West Windsor Township, Mercer County, New Jersey, United States",40.321632,-74.619494,0.559973
7,Mamoun's Falafel,"Mamoun's Falafel, Condict Street, New Brunswick, Middlesex County, New Jersey, 08901, United States",40.497734,-74.449132,0.301000
8,"Palmer Square, Princeton","Palmer Square, Princeton, Mercer County, New Jersey, 08542, United States",40.349936,-74.661056,0.410000
9,Marquand Park,"Marquand Park, Princeton, Mercer County, New Jersey, 08540-4810, United States",40.341389,-74.670833,0.481312


## Creating Interactive Maps
We're going to usa a Python library called Folium to visualize our coordinate points on maps from OpenStreet Maps. 

### Install Folium

In [72]:
!pip install folium

     |████████████████████████████████| 94 kB 3.0 MB/s eta 0:00:011


### Import Folium

In [5]:
import folium

### Make a Base Map
First, we need to establish a base map. This is where we’ll map our geocoded Princeton locations. To do so, we’re going to call folium.Map()and enter the general latitude/longitude coordinates of the Ithaca area at a particular zoom.

(To find latitude/longitude coordintes for a particular location, you can use Google Maps, as described here.)

In [74]:
princeton_map = folium.Map(location=[40.35, -74.65], zoom_start=14)
princeton_map

### Add a Marker

Adding a marker to a map is easy with Folium! We’ll simply call `folium.Marker()` at a particular lat/lon, enter some text to display when the marker is clicked on, and then add it to our base map.

In [75]:
folium.Marker(location=[40.35, -74.65],popup="Intro to Digital_Humanities").add_to(princeton_map)
princeton_map


## Add Markers from dataframe

To add markers for every location in our Pandas dataframe, we can make a Python function and `.apply()` it to every row in the dataframe.

In [76]:
def create_map_markers(row, map_name):
    folium.Marker(location=[row['lat'], row['lon']], popup=row['place']).add_to(map_name)

Before we apply this function to our dataframe, we’re going to drop any locations that were “Not Found” (which would cause folium.Marker() to return an error).

In [77]:
found_princeton_locations = princeton_df[princeton_df['address'] != "Not Found"]

In [78]:
found_princeton_locations.apply(create_map_markers, map_name=princeton_map, axis='columns')
princeton_map

### To save map as an HTML file

In [79]:
princeton_map.save("Princeton-map.html")

---


## Your Turn!
- Create a CSV file with your group.
- Use GeoPy to extract the coordinates from that CSV
- Plot it on a map

In [33]:
### Your code here

In [34]:
### Your code here

In [35]:
### Your code here